<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/27_Joins_con_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 27. Joins con Spark


In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_timestamp

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Un Join es una operación que permite combinar datos de dos o más tablas en una sola, basándose en una relación entre ellas. Usualmente, esta relación se establece a través de una columna en común entre las tablas.

El Join permite acceder y manipular datos que están distribuidos en varias tablas de manera conjunta, obteniendo así resultados más complejos y completos.

Vamos a leer tres tablas; estas se encuentran en este [enlace de Kaggle](https://www.kaggle.com/datasets/svbstan/sales-product-and-customer-insight-repository)

In [ ]:
customer = spark.read.csv('customer_profile_dataset.csv', header=True)
customer.show(5)

+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+
|customer_id|first_name|last_name|gender|      date_of_birth|               email|phone_number|        signup_date|      address|       city|state|zip_code|
+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+
|          1|    Robert|    Smith|Female|1994-06-14 21:40:27|jane.davis1@mail.com|634-106-4981|2016-10-16 17:23:25| 8465 Main St|San Antonio|   CA|   35566|
|          2|     Emily|   Garcia|Female|1989-09-21 17:56:31|robert.williams2@...|386-635-5998|2021-04-04 14:24:06|  305 Main St|   New York|   AZ|   23187|
|          3|   Jessica|    Brown|  Male|1984-01-21 21:43:13|emily.davis3@mail...|627-341-5213|2018-04-22 04:51:57|  5725 Oak St|    Chicago|   AZ|   99188|
|          4|   Michael|    Brown|  Male|1986-02-06 13:09:

In [ ]:
products = spark.read.csv('products_dataset.csv', header=True)
products.show(5)

+----------+------------+--------+--------------+------+--------------------+
|product_id|product_name|category|price_per_unit| brand| product_description|
+----------+------------+--------+--------------+------+--------------------+
|         1|      Butter|   Dairy|         28.58|BrandB|Description for Rice|
|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|         3|        Milk|   Meats|         26.52|BrandE|Description for B...|
|         4|      Banana|  Grains|         26.12|BrandB|Description for A...|
|         5|        Rice|  Fruits|         21.94|BrandD|Description for B...|
+----------+------------+--------+--------------+------+--------------------+
only showing top 5 rows



In [ ]:
purchase = spark.read.csv('purchase_history_dataset.csv', header=True)
purchase.show(5)

+-----------+-----------+----------+-------------------+--------+------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|
+-----------+-----------+----------+-------------------+--------+------------------+
|          1|          1|        42|2018-04-15 14:08:01|       3| 37.64207365077783|
|          2|          1|       138|2022-07-10 23:33:47|       4| 70.24710587172727|
|          3|          1|       403|2021-12-31 03:53:33|       3| 89.16889585975464|
|          4|          1|       193|2017-01-14 01:25:11|       2| 59.70505931112876|
|          5|          1|        26|2018-04-06 11:01:06|       3|101.77886387225126|
+-----------+-----------+----------+-------------------+--------+------------------+
only showing top 5 rows



Hacer un Join en Spark es parecido a como lo haríamos en SQL...o Pandas, dplyr y la mayoría de las librerías que has usado para manipular datos.

```
df.join(df2, df.column == df2.column, how={'inner'})
```




In [11]:
print((customer.count(), len(customer.columns)))

(1000, 12)


In [12]:
print((products.count(), len(products.columns)))

(500, 6)


In [9]:
print((purchase.count(), len(purchase.columns)))

(10308, 6)


## Tipos de Joins

### INNER JOIN

Mantiene los datos en común en ambas tablas.

In [17]:
purchase_inner_products = purchase.join(products, purchase.product_id == products.product_id, how='inner')
purchase_inner_products.show()

+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|          1|          1|        42|2018-04-15 14:08:01|       3| 37.64207365077783|        42|        Milk|  Fruits|         21.63|BrandD|Description for B...|
|          2|          1|       138|2022-07-10 23:33:47|       4| 70.24710587172727|       138|       Bread|  Grains|         25.98|BrandE|Description for B...|
|          3|          1|       403|2021-12-31 03:53:33|       3| 89.16889585975464|       403|       Bread|   Meats|         24.01|BrandD|Description for B...|
|          4|          1|       19

In [18]:
print((purchase_inner_products.count(), len(purchase_inner_products.columns)))

(10308, 12)


Como TODOS los productos de mi Dataset aparecían en mi tabla de compras y viceversa; además, *purchase* es una tabla más grande que *products*:

$=> n(purchase) =  n(purchase\_inner\_products) $

Pero a ver, vamos a filtrar antes mi tabla de purchase y mi tabla de productos.

In [29]:
purchase2 = purchase.withColumn('purchase_date', to_timestamp(col('purchase_date'), 'yyyy-MM-dd HH:mm:ss'))\
                    .filter(purchase["purchase_date"] > lit("2022-01-01 00:00:00"))

purchase2.count()

4849

In [31]:
products2 = products.filter(products.category == 'Meats')
products2.count()

136

In [32]:
purchase2_inner_products2 = purchase2.join(products2, purchase.product_id == products2.product_id, how='inner')
purchase2_inner_products2.show()

+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|          7|          1|       192|2023-09-21 08:52:12|       5|201.98834253689253|       192|      Butter|   Meats|         26.46|BrandE|Description for P...|
|         14|          2|        55|2022-12-19 10:28:44|       1| 42.61748689307072|        55|       Apple|   Meats|         23.58|BrandD|Description for A...|
|         35|          4|       305|2022-07-18 04:54:39|       1|14.453941727047198|       305|       Pasta|   Meats|          6.06|BrandD|Description for P...|
|         44|          4|       19

In [33]:
purchase2_inner_products2.count()

1320

Ahora sí; el número de la tabla con el INNER JOIN **NO** coincide con ninguna de mis dos tablas, puesto que sólo mantuve los registros en común.

### RIGHT / LEFT JOIN

En un RIGHT JOIN mantengo todos los registros de la tabla derecha y sólo aquellos que también coinciden con los de la derecha. Por lo tanto, mientras haga el cruce con una columna que tenga valores únicos, el número de hileras de la tabla resultante es igual al de la tabla derecha.

En un LEFT JOIN sucede justo lo contrario.

In [34]:
purchase2_right_products2 = purchase2.join(products2, purchase.product_id == products2.product_id, how='right')
purchase2_right_products2.show()

+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|       5799|        568|         2|2023-02-06 01:22:45|       5|140.47481851783164|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|       5724|        561|         2|2022-07-21 18:15:54|       1|44.977080921698125|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|       4249|        413|         2|2022-07-30 15:05:28|       5|165.27179735866832|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|       4069|        393|         

In [35]:
purchase2_right_products2.count()

1320

Coincidió con el número de elementos en el INNER JOIN porque todos los elementos de mi tabla derecha (products2) aparecían también en mi tabla izquierda (purchase2)

In [36]:
purchase2_left_products2 = purchase2.join(products2, purchase.product_id == products2.product_id, how='left')
purchase2_left_products2.show()

+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|          2|          1|       138|2022-07-10 23:33:47|       4| 70.24710587172727|      NULL|        NULL|    NULL|          NULL|  NULL|                NULL|
|          7|          1|       192|2023-09-21 08:52:12|       5|201.98834253689253|       192|      Butter|   Meats|         26.46|BrandE|Description for P...|
|          9|          2|       218|2023-11-03 08:21:19|       5|230.93891243311947|      NULL|        NULL|    NULL|          NULL|  NULL|                NULL|
|         10|          2|       17

In [37]:
purchase2_left_products2.count()

4849

### OUTER JOINS
Mantienen todos los registros, independientemente de si coinciden o no.

Estos, a su vez, pueden ser LEFT OUTER, RIGHT OUTER o FULL OUTER.

In [40]:
purchase2_rightouter_products2 = purchase2.join(products2, purchase.product_id == products2.product_id, how='rightouter')

purchase2_rightouter_products2.show()

+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|       5799|        568|         2|2023-02-06 01:22:45|       5|140.47481851783164|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|       5724|        561|         2|2022-07-21 18:15:54|       1|44.977080921698125|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|       4249|        413|         2|2022-07-30 15:05:28|       5|165.27179735866832|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|       4069|        393|         

In [42]:
purchase2_rightouter_products2.count()

1320

In [43]:
purchase2_fullouter_products2 = purchase2.join(products2, purchase.product_id == products2.product_id, how='fullouter')

purchase2_fullouter_products2.show()

+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+----------+------------+--------+--------------+------+--------------------+
|        905|         87|         1|2022-08-02 05:58:01|       2| 90.00597143593103|      NULL|        NULL|    NULL|          NULL|  NULL|                NULL|
|       1783|        173|         1|2023-02-21 15:18:31|       2|35.442343007734145|      NULL|        NULL|    NULL|          NULL|  NULL|                NULL|
|       6676|        656|         1|2022-05-07 02:42:26|       5|192.64166873691394|      NULL|        NULL|    NULL|          NULL|  NULL|                NULL|
|       6986|        685|         

In [44]:
purchase2_fullouter_products2.count()

4849

Ejercicios; contesta desarrollando el correspondiente código en Pyspark:

1. Responde, ¿cuántos clientes llamados "Robert" (nota cómo hay *Males* y *Females*), compraron algún producto lácteo (Dairy) en 2022 ?

2. Eres empleado de *BrandB*. ¿En cuáles ciudades has vendido una mayor cantidad? (total_amount)

3. ¿De cuánto es la mayor cantidad (quantity) que ha sido comprado por algún hombre O cuyo producto sea pan (Bread)?